## Hipótesis

- Tiempo de resolución: ¿Son los tickets con mayor severidad y prioridad los que se resuelven antes?

- Satisfacción del cliente: ¿Están los clientes con problemas de acceso y login más contentos que los que tienen problemas de software?

- Desempeño del agente: ¿Cuáles son los empleados que más tickets atienden? ¿Son los empleados que atienden más tickets los que tardan más en solucionar los problemas de los clientes? ¿Y la media de la satisfacción de los clientes es menor en aquellos empleados que atienden más solicitudes?

- Cambios en el tiempo: ¿Hay alguna diferencia entre a lo largo del tiempo entre las distintas categorías de solicitudes de tickets que tenemos?

In [1]:
import pandas as pd

In [6]:
import os

In [9]:
df = pd.read_csv("datos/historico_tickets_limpio_pandas.csv", index_col=0)

## Objetivos

1. Haced una exploración inicial del dataframe:

- ¿Cuál es el número de filas y columnas?

In [10]:
df.head()

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
0,GDDENR-5042564453,2016-07-13,1735,4,Acceso/Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0,5,2º semestre,2016,Julio
1,GDDENR-8042508060,2016-05-18,1566,10,Acceso/Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0,5,1 semestre,2016,Mayo
2,GDDESR-1342539995,2016-06-18,569,29,Sistema,Problema,0 - Sin Clasificar,0 - Sin Asignar,3,5,1 semestre,2016,Junio
3,GDDTSR-5942488006,2016-04-28,320,40,Sistema,Solicitud,0 - Sin Clasificar,0 - Sin Asignar,9,5,1 semestre,2016,Abril
4,GDLEER-0042524120,2016-06-03,1842,31,Software,Problema,2 - Normal,0 - Sin Asignar,0,5,1 semestre,2016,Junio


In [11]:
df.shape #97386 filas y 13 columnas

(97386, 13)

- ¿Cuál es el porcentaje de valores nulos en cada columna? Presenta los resultados en un dataframe, donde las columnas sean el nombre de las variables y el porcentaje de nulos.

In [16]:
df_nulos = (df.isnull().sum()*100/df.shape[0]).reset_index()
df_nulos

,index,0
0,id_ticket,0.0
1,fecha,0.0
2,id_empleado,0.0
3,id_agente,0.0
4,categoría,0.0
5,tipo,0.0
6,severidad,0.0
7,prioridad,0.0
8,días_resolución,0.0
9,satisfacción,0.0


- ¿Qué tipos de datos hay en cada columna?

In [17]:
df.dtypes #la fecha se podría cambiar a datetime

id_ticket          object
fecha              object
id_empleado         int64
id_agente           int64
categoría          object
tipo               object
severidad          object
prioridad          object
días_resolución     int64
satisfacción        int64
semestre           object
año                 int64
mes                object
dtype: object

- Extraed la información general del dataframe.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97386 entries, 0 to 29087
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_ticket        97386 non-null  object
 1   fecha            97386 non-null  object
 2   id_empleado      97386 non-null  int64 
 3   id_agente        97386 non-null  int64 
 4   categoría        97386 non-null  object
 5   tipo             97386 non-null  object
 6   severidad        97386 non-null  object
 7   prioridad        97386 non-null  object
 8   días_resolución  97386 non-null  int64 
 9   satisfacción     97386 non-null  int64 
 10  semestre         97386 non-null  object
 11  año              97386 non-null  int64 
 12  mes              97386 non-null  object
dtypes: int64(5), object(8)
memory usage: 10.4+ MB


2. Cread dos dataframes nuevos, uno que incluya solo las variables categóricas y otro que incluya solo las variables numéricas.

- ¿Cuáles son los principales estádisticos de los dataframes?

In [22]:
df_cat = df.select_dtypes(include = 'object')
df_cat.head(1)

,id_ticket,fecha,categoría,tipo,severidad,prioridad,semestre,mes
0,GDDENR-5042564453,2016-07-13,Acceso/Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,2º semestre,Julio


In [23]:
df_num = df.select_dtypes(include = 'int64')
df_num.head(1)

,id_empleado,id_agente,días_resolución,satisfacción,año
0,1735,4,0,5,2016


In [25]:
df_cat.describe().T

,count,unique,top,freq
id_ticket,97386,97386,GDDENR-5042564453,1
fecha,97386,1827,2020-02-20,105
categoría,97386,4,Sistema,38961
tipo,97386,2,Solicitud,73138
severidad,97386,5,2 - Normal,88555
prioridad,97386,4,3 - Alta,35511
semestre,97386,2,2º semestre,49876
mes,97386,12,Octubre,8487


In [27]:
df_num.describe().T

,count,mean,std,min,25%,50%,75%,max
id_empleado,97386.0,999.160639,577.407222,1.0,500.0,999.0,1499.0,2000.0
id_agente,97386.0,25.468856,14.449455,1.0,13.0,26.0,38.0,50.0
días_resolución,97386.0,4.552749,4.365432,0.0,0.0,4.0,7.0,21.0
satisfacción,97386.0,4.100990,1.257093,1.0,4.0,5.0,5.0,5.0
año,97386.0,2018.399164,1.393095,2016.0,2017.0,2019.0,2020.0,2020.0


- Extraed los valores únicos.

In [30]:
for col in df_cat.columns: #omitimos las columnas con 'id' porque todos son valores únicos

    if 'id' in col:
        pass
    else:

        print("------------------------------------------------------------", "\n")
        print(f"En la columna {col} tenemos los siguientes valores unicos")
        print(pd.DataFrame(df_cat[col].value_counts()))
        print("\n", "------------------------------------------------------------", "\n")

------------------------------------------------------------ 

En la columna fecha tenemos los siguientes valores unicos
            fecha
2020-02-20    105
2020-12-16    105
2020-12-31    103
2020-03-14    103
2020-06-14    102
...           ...
2016-08-21     22
2016-03-04     22
2016-04-14     21
2016-07-08     18
2016-11-17     17

[1827 rows x 1 columns]

 ------------------------------------------------------------ 

------------------------------------------------------------ 

En la columna categoría tenemos los siguientes valores unicos
              categoría
Sistema           38961
Acceso/Login      29165
Software          19543
Hardware           9717

 ------------------------------------------------------------ 

------------------------------------------------------------ 

En la columna tipo tenemos los siguientes valores unicos
            tipo
Solicitud  73138
Problema   24248

 ------------------------------------------------------------ 

---------------------------

In [31]:
for col in df_num.columns: #omitimos las columnas con 'id' porque todos son valores únicos

    if 'id' in col:
        pass
    else:

        print("------------------------------------------------------------", "\n")
        print(f"En la columna {col} tenemos los siguientes valores unicos")
        print(pd.DataFrame(df_num[col].value_counts()))
        print("\n", "------------------------------------------------------------", "\n")

------------------------------------------------------------ 

En la columna días_resolución tenemos los siguientes valores unicos
    días_resolución
0             25049
1              9267
5              8779
6              7793
7              6578
2              6454
3              6194
4              4911
8              4841
10             3895
9              3732
11             1729
13             1711
14             1566
12             1555
15             1357
16             1165
17              553
19              130
18              123
21                2
20                2

 ------------------------------------------------------------ 

------------------------------------------------------------ 

En la columna satisfacción tenemos los siguientes valores unicos
   satisfacción
5         50722
4         27529
1          9888
3          7271
2          1976

 ------------------------------------------------------------ 

-------------------------------------------------------

#### BONUS 

Presenta los resultados en un dataframe.

Donde variable es cada una de las variables categóricas que tenemos en el dataset y n_unicos es una lista con los nombres de los valores únicos de cada variable.

In [48]:
valores_unicos = {}

In [49]:
for col in df_cat.columns:

    if 'id' in col:
        pass
    elif 'fecha' in col:
        pass

    else:
        
        valores_unicos[col] = df_cat[col].unique()

df_unicos = pd.Series(valores_unicos).reset_index()


In [50]:
df_unicos = df_unicos.rename(columns = {'index': 'variable', 0 : 'n_unicos'})
df_unicos

,variable,n_unicos
0,categoría,"[Acceso/Login, Sistema, Software, Hardware]"
1,tipo,"[Problema, Solicitud]"
2,semestre,"[2º semestre, 1 semestre]"
3,mes,"[Julio, Mayo, Junio, Abril, Agosto, Marzo, Ene..."


In [51]:
df.sample(10)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
4848,GWLENR-4143147806,2018-02-16,1200,38,Acceso/Login,Problema,2 - Normal,1 - Baja,1,5,1 semestre,2018,Febrero
5665,GWLTER-2543703020,2019-08-26,892,30,Hardware,Solicitud,2 - Normal,1 - Baja,14,5,2º semestre,2019,Agosto
7756,GWLTER-2443986734,2020-06-04,1143,7,Hardware,Solicitud,2 - Normal,1 - Baja,19,5,1 semestre,2020,Junio
16793,THLTER-9443179009,2018-03-20,1800,20,Software,Solicitud,2 - Normal,3 - Alta,4,5,1 semestre,2018,Marzo
16154,KWLTNR-6144099311,2020-09-25,811,23,Acceso/Login,Solicitud,2 - Normal,1 - Baja,1,3,2º semestre,2020,Septiembre
6591,KDLTNR-8043435464,2018-12-01,726,14,Acceso/Login,Solicitud,2 - Normal,0 - Sin Asignar,0,1,2º semestre,2018,Diciembre
11744,SDLTNR-7043141753,2018-02-10,1655,47,Acceso/Login,Solicitud,2 - Normal,0 - Sin Asignar,0,4,1 semestre,2018,Febrero
12072,KWLTSR-3443805096,2019-12-06,469,50,Sistema,Solicitud,2 - Normal,1 - Baja,14,3,2º semestre,2019,Diciembre
11352,SMLTSR-8642879613,2017-05-24,1861,46,Sistema,Solicitud,2 - Normal,2 - Media,12,5,1 semestre,2017,Mayo
2206,GHLTSR-7742601138,2016-08-19,3,21,Sistema,Solicitud,2 - Normal,3 - Alta,7,5,2º semestre,2016,Agosto


## Conclusiones

- ¿Hay alguna columna que no tenga sentido su tipo de datos?

- ¿Hay alguna columna que no tenga sentido su nombre?

- ¿Hay alguna columna que no tenga sentido su contenido?

- ¿Hay alguna columna que tenga demasiadas categorías o valores que nos pueda hacer difícil trabajar con ella?

1. Hay columnas con demasiados valores únicos que dificultan el trabajo con ellos, como por ejemplo las columnas 'id', en las que cada valor es único y no es muy útil trabajar con los únicos. 
2. Se puede cambiar el tipo de dato de la columna 'fecha', pues está en tipo object y lo más conveniente sería pasarlo a formato datetime, para trabajar más fácilmente con fechas.

3. Se puede concluir que hay más solicitudes que problemas, por lo que la resolución de problemas está más controlada y éstos no sobrecargan las peticiones.  
4. En el segundo semestre hay un ligero aumento de peticiones.
5. Hay más peticiones relacionadas con el sistema que con las otras categorías.
